In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,31331
2,Huelva,Confirmados PDIA 14 días,1422
3,Huelva,Tasa PDIA 14 días,"277,10115556248417"
4,Huelva,Confirmados PDIA 7 días,733
5,Huelva,Total Confirmados,31511
6,Huelva,Curados,28904
7,Huelva,Fallecidos,383


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  31331.0
<ipython-input-7-31b732756b19>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  8320.0
<ipython-input-8-e81bfb40e710>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

<ipython-input-9-f09dd2050cc1>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 519 personas en los últimos 7 días 

Un positivo PCR cada 295 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,31331.0,733.0,1422.0,513170.0,142.837656,277.101156,174.0
Huelva-Costa,18028.0,378.0,717.0,289548.0,130.548303,247.627336,92.0
Huelva (capital),8320.0,277.0,487.0,143837.0,192.579100,338.577696,68.0
Condado-Campiña,10188.0,281.0,578.0,156231.0,179.861871,369.964988,59.0
Sierra de Huelva-Andévalo Central,2767.0,71.0,121.0,67391.0,105.355315,179.549198,21.0
Palos de la Frontera,662.0,29.0,53.0,11742.0,246.976665,451.371146,18.0
Valverde del Camino,576.0,37.0,56.0,12750.0,290.196078,439.215686,14.0
Moguer,1289.0,45.0,102.0,21867.0,205.789546,466.456304,13.0
Aljaraque,1093.0,35.0,64.0,21474.0,162.987799,298.034833,8.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Villarrasa,172.0,6.0,19.0,2211.0,271.370421,859.339665,1.0
Niebla,235.0,16.0,26.0,4116.0,388.726919,631.681244,2.0
Almonte,1817.0,65.0,128.0,24507.0,265.230342,522.299751,2.0
Palma del Condado (La),1149.0,16.0,52.0,10801.0,148.134432,481.436904,3.0
Calañas,191.0,2.0,13.0,2768.0,72.254335,469.653179,1.0
Moguer,1289.0,45.0,102.0,21867.0,205.789546,466.456304,13.0
Palos de la Frontera,662.0,29.0,53.0,11742.0,246.976665,451.371146,18.0
Valverde del Camino,576.0,37.0,56.0,12750.0,290.196078,439.215686,14.0
Manzanilla,99.0,2.0,9.0,2118.0,94.428706,424.929178,1.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Calañas,191.0,2.0,13.0,2768.0,72.254335,469.653179,1.0,0.153846
Beas,210.0,1.0,6.0,4341.0,23.036167,138.217001,0.0,0.166667
Manzanilla,99.0,2.0,9.0,2118.0,94.428706,424.929178,1.0,0.222222
Ayamonte,1317.0,5.0,19.0,21104.0,23.692191,90.030326,0.0,0.263158
Lepe,2127.0,7.0,24.0,27880.0,25.107604,86.083214,0.0,0.291667
Punta Umbría,808.0,11.0,37.0,15355.0,71.637903,240.963855,3.0,0.297297
Palma del Condado (La),1149.0,16.0,52.0,10801.0,148.134432,481.436904,3.0,0.307692
Villarrasa,172.0,6.0,19.0,2211.0,271.370421,859.339665,1.0,0.315789
Alosno,167.0,1.0,3.0,3933.0,25.425884,76.277651,0.0,0.333333
